In [24]:
import numpy as np
import pandas as pd
from datetime import datetime

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F


Importamos como dataframes los datos que vamos a usar.

In [2]:
weather_columns = ['Station', 'Date', 'Tmax', 'Tmin', 'Tavg', 'Depart', 'DewPoint', 'WetBulb', 'Heat', 'Cool', 'Sunrise', 'Sunset', 'PrecipTotal', 'StnPressure', 'SeaLevel', 'ResultSpeed', 'ResultDir', 'AvgSpeed']
spray_columns = ['Date','Latitude','Longitude']
train_columns = ['Date', 'Species', 'Latitude', 'Longitude', 'NumMosquitos']
test_columns = ['Id', 'Date', 'Species', 'Latitude', 'Longitude']

weather_df = pd.read_csv('../input/weather.csv')[weather_columns]
spray_df = pd.read_csv('../input/spray.csv')[spray_columns]
train_df = pd.read_csv('../input/train.csv')[train_columns]
test_df = pd.read_csv('../input/test.csv')[test_columns]
y_df = pd.read_csv('../input/train.csv')['WnvPresent']

Definimos funciones y objetos que nos ayudarán a darle a todos los dataframes los tipos de datos convenientes.

In [3]:
def to_float(x):
    if x == "M":
        return None
    return float(x)

def precip(text):
    TRACE = 1e-3
    text = text.strip()
    if text == "M":
        return None
    if text == "-":
        return None
    if text == "T":
        return TRACE
    return float(text)

species_map = {
    "CULEX RESTUANS": "100000",
    "CULEX TERRITANS": "010000",
    "CULEX PIPIENS": "001000",
    "CULEX PIPIENS/RESTUANS": "101000",
    "CULEX ERRATICUS": "000100",
    "CULEX SALINARIUS": "000010",
    "CULEX TARSALIS": "000001",
    "UNSPECIFIED CULEX": "001000",
}

weather_float = ['Tmax', 'Tmin', 'Tavg', 'DewPoint', 'WetBulb', 'Depart', 'ResultSpeed', 'ResultDir', 'AvgSpeed', 'StnPressure', 'SeaLevel']
weather_precip = ["Heat", "Cool", "Sunrise", "Sunset", "PrecipTotal"]

Por cada dataframe usamos las funciones y objetos de la celda anterior para que sus valores sean del tipo necesario.

In [4]:
weather_df["Date"] = weather_df["Date"].apply(lambda x : datetime.strptime(x, "%Y-%m-%d").timestamp())

for column in weather_df.columns:
    if column in weather_float:
        weather_df[column] = weather_df[column].apply(to_float)
    elif column in weather_precip:
        weather_df[column] = weather_df[column].apply(precip)

In [5]:
spray_df["Date"] = spray_df["Date"].apply(lambda x : datetime.strptime(x, "%Y-%m-%d").timestamp())

In [6]:
train_df["Date"] = train_df["Date"].apply(lambda x : datetime.strptime(x, "%Y-%m-%d").timestamp())
train_df["Species"] = train_df["Species"].map(species_map).astype("int64")

In [7]:
test_df["Date"] = test_df["Date"].apply(lambda x : datetime.strptime(x, "%Y-%m-%d").timestamp())
test_df["Species"] = test_df["Species"].map(species_map).astype("int64")

Por cada registro de las trampas de mosquitos, encontraremos la estación meteorológica más cercana para asociarle información del tiempo de ese día.

In [8]:
# # Radio de la Tierra es aprox. 6378.137km
# def semiverseno(coord1,coord2):
#     lat1, lon1, lat2, lon2 = map(np.radians, list(coord1 + coord2))
#     dlat = lat2 - lat1
#     dlon = lon2 - lon1
#     a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
#     b  = 2 * np.arcsin(np.sqrt(a))
#     return b* 6378.137

from geopy.distance import geodesic

# Estación 1: 41.995 -87.933
# Estación 2: 41.786 -87.752
def find_station(lat, lon):
    d1 = geodesic((41.995, -87.933),(lat, lon))
    d2 = geodesic((41.786, -87.752),(lat, lon))

    if d1 < d2:
        return 1 
    return 2

In [9]:
Station = []

for i in range(10506):
    lat = train_df.iloc[i]["Latitude"]
    lon = train_df.iloc[i]["Longitude"]
    station = find_station(lat,lon)

    Station.append(station)

train_df['Station'] = Station

Con lo anterior creamos el dataframe final.

In [10]:
x_df = pd.merge(train_df, weather_df, on=["Date", "Station"], sort=False)
x_df.drop('Station', axis=1, inplace=True)

Por último, usamos la información en la tabla de rociado (spray_df) para indicar si el día en el que se recogió la muestra de la trampa de mosquitos, se roció dentro de un radio de 10 metros de la ubicación de la trampa.

In [11]:
Spray = []

for i in range(10506):
    loc = (train_df.iloc[i]["Latitude"], train_df.iloc[i]["Longitude"])
    temp_df = spray_df[spray_df['Date'] == x_df.iloc[i]['Date']]
    temp_spray = []
    for index, row in temp_df.iterrows():
        if geodesic(loc,(row["Latitude"], row['Longitude'])).meters < 10:
            temp_spray.append(1)
        else:
            temp_spray.append(0)
    if any(temp_spray):
        Spray.append(1)
    else:
        Spray.append(0)

In [12]:
x_df['Spray'] = Spray

In [13]:
x_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10506 entries, 0 to 10505
Data columns (total 22 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date          10506 non-null  float64
 1   Species       10506 non-null  int64  
 2   Latitude      10506 non-null  float64
 3   Longitude     10506 non-null  float64
 4   NumMosquitos  10506 non-null  int64  
 5   Tmax          10506 non-null  float64
 6   Tmin          10506 non-null  float64
 7   Tavg          10506 non-null  float64
 8   Depart        3298 non-null   float64
 9   DewPoint      10506 non-null  float64
 10  WetBulb       10470 non-null  float64
 11  Heat          10506 non-null  float64
 12  Cool          10506 non-null  float64
 13  Sunrise       3298 non-null   float64
 14  Sunset        3298 non-null   float64
 15  PrecipTotal   10482 non-null  float64
 16  StnPressure   10470 non-null  float64
 17  SeaLevel      10506 non-null  float64
 18  ResultSpeed   10506 non-nu

In [14]:
x_df.describe()

,Date,Species,Latitude,Longitude,NumMosquitos,Tmax,Tmin,Tavg,Depart,DewPoint,...,Cool,Sunrise,Sunset,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed,Spray
count,1.050600e+04,10506.000000,10506.000000,10506.000000,10506.000000,10506.000000,10506.000000,10506.000000,3298.000000,10506.000000,...,10506.000000,3298.000000,3298.000000,10482.000000,10470.000000,10506.000000,10506.000000,10506.000000,10506.000000,10506.000000
mean,1.267690e+09,72232.245003,41.841139,-87.699908,12.853512,81.928707,63.991053,73.187512,2.064585,59.653912,...,9.054826,464.422680,1874.785021,0.137374,29.292056,29.958250,5.918142,17.688369,7.455930,0.000190
std,7.400291e+07,44801.136822,0.112742,0.096514,16.133816,8.285836,7.585434,7.506371,6.534137,7.838916,...,5.783582,46.106781,63.889569,0.341398,0.117427,0.117162,2.902601,9.136063,2.521806,0.013797
min,1.180415e+09,1.000000,41.644612,-87.930995,1.000000,57.000000,41.000000,50.000000,-12.000000,38.000000,...,0.000000,416.000000,1720.000000,0.000000,28.890000,29.590000,0.100000,1.000000,2.100000,0.000000
25%,1.187932e+09,1000.000000,41.732984,-87.760070,2.000000,78.000000,60.000000,70.000000,-3.000000,54.000000,...,5.000000,426.000000,1835.000000,0.000000,29.210000,29.880000,3.900000,9.000000,5.800000,0.000000
50%,1.249621e+09,100000.000000,41.846283,-87.694991,5.000000,83.000000,66.000000,75.000000,3.000000,60.000000,...,10.000000,445.000000,1911.000000,0.000000,29.290000,29.960000,5.500000,19.000000,7.100000,0.000000
75%,1.316149e+09,101000.000000,41.954690,-87.627796,17.000000,88.000000,70.000000,79.000000,7.000000,66.000000,...,14.000000,512.000000,1927.000000,0.090000,29.380000,30.050000,7.800000,24.000000,9.200000,0.000000
max,1.380172e+09,101000.000000,42.017430,-87.531635,50.000000,97.000000,79.000000,87.000000,18.000000,73.000000,...,22.000000,557.000000,1931.000000,3.970000,29.650000,30.330000,15.400000,36.000000,16.300000,1.000000


In [15]:
x_df.head()

,Date,Species,Latitude,Longitude,NumMosquitos,Tmax,Tmin,Tavg,Depart,DewPoint,...,Cool,Sunrise,Sunset,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed,Spray
0,1.180415e+09,101000,41.954690,-87.800991,1,88.0,60.0,74.0,10.0,58.0,...,9.0,421.0,1917.0,0.0,29.39,30.11,5.8,18.0,6.5,0
1,1.180415e+09,100000,41.954690,-87.800991,1,88.0,60.0,74.0,10.0,58.0,...,9.0,421.0,1917.0,0.0,29.39,30.11,5.8,18.0,6.5,0
2,1.180415e+09,100000,41.994991,-87.769279,1,88.0,60.0,74.0,10.0,58.0,...,9.0,421.0,1917.0,0.0,29.39,30.11,5.8,18.0,6.5,0
3,1.180415e+09,101000,41.974089,-87.824812,1,88.0,60.0,74.0,10.0,58.0,...,9.0,421.0,1917.0,0.0,29.39,30.11,5.8,18.0,6.5,0
4,1.180415e+09,100000,41.974089,-87.824812,4,88.0,60.0,74.0,10.0,58.0,...,9.0,421.0,1917.0,0.0,29.39,30.11,5.8,18.0,6.5,0


In [16]:
class CustomDataset(Dataset):
    def __init__(self, dataframe):
        self.date = dataframe["Date"]
        self.species = dataframe["Species"]
        self.latitude = dataframe["Latitude"]
        self.longitude = dataframe["Longitude"]
        self.numMosq = dataframe["NumMosquitos"]
        self.tmax = dataframe["Tmax"]
        self.tmin = dataframe["Tmin"]
        self.tavg = dataframe["Tavg"]
        self.depart = dataframe["Depart"]
        self.dewpoint = dataframe["DewPoint"]
        self.wetBulb = dataframe["WetBulb"]
        self.heat = dataframe["Heat"]
        self.cool = dataframe["Cool"]
        self.sunrise = dataframe["Sunrise"]
        self.sunset = dataframe["Sunset"]
        self.precipTotal = dataframe["PrecipTotal"]
        self.stnPressure = dataframe["StnPressure"]
        self.seaLevel = dataframe["SeaLevel"]
        self.resultSpeed = dataframe["ResultSpeed"]
        self.resultDir = dataframe["ResultDir"]
        self.avgSpeed = dataframe["AvgSpeed"]
        self.spray = dataframe["Spray"]

    def __len__(self):
        return len(self.date)

    def __getitem__(self, idx):
        date = self.date.iloc[idx]
        species = self.species.iloc[idx]
        latitude = self.latitude.iloc[idx]
        longitude = self.longitude.iloc[idx]
        numMosq = self.numMosq.iloc[idx]
        tmax = self.tmax.iloc[idx]
        tmin = self.tmin.iloc[idx]
        tavg = self.tavg.iloc[idx]
        depart = self.depart.iloc[idx]
        dewpoint = self.dewpoint.iloc[idx]
        wetBulb = self.wetBulb.iloc[idx]
        heat = self.heat.iloc[idx]
        cool = self.cool.iloc[idx]
        sunrise = self.sunrise.iloc[idx]
        sunset = self.sunset.iloc[idx]
        precipTotal = self.precipTotal.iloc[idx]
        stnPressure = self.stnPressure.iloc[idx]
        seaLevel = self.seaLevel.iloc[idx]
        resultSpeed = self.resultSpeed.iloc[idx]
        resultDir = self.resultDir.iloc[idx]
        avgSpeed = self.avgSpeed.iloc[idx]
        spray = self.spray.iloc[idx]
        return date, species, latitude, longitude, numMosq, tmax, tmin, tavg, depart, dewpoint, wetBulb, heat, cool, sunrise, sunset, precipTotal, stnPressure, seaLevel, resultSpeed, resultDir, avgSpeed, spray

In [17]:
wnv = CustomDataset(x_df)
y_df = torch.tensor(y_df)

In [18]:
train_dataloader = DataLoader(wnv, batch_size=8006, shuffle=True)
test_dataloader = DataLoader(wnv, batch_size=2500, shuffle=True)

In [25]:
class Model(nn.Module):
    def __init__(self, in_features=22, h1=15, h2=15, out_features=1):
        super().__init__()
        self.fc1 = nn.Linear(in_features,h1)    # input layer
        self.fc2 = nn.Linear(h1, h2)            # hidden layer
        self.out = nn.Linear(h2, out_features)  # output layer
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.out(x)
        return x

In [26]:
model = Model()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.05)

In [29]:
x_train = torch.tensor(train_dataloader)

RuntimeError: Could not infer dtype of DataLoader

In [27]:
epochs = 100
losses = []

for i in range(epochs):
    i+=1
    y_pred = model.forward(train_dataloader)
    loss = criterion(y_pred,y_df)
    losses.append(loss)

    if i%10 == 1:
        print(f'epoch: {i}\tloss: {loss.item():10.8f}')

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

TypeError: linear(): argument 'input' (position 1) must be Tensor, not DataLoader